In [2]:
from web_parser import AnswerParser, QuestionParser
import multiprocessing
from multiprocessing import Queue, Process
from selenium import webdriver
from pathlib import Path
import os
import yaml
from config import drivers, nr_answer_parsers, get_db_path, get_vcs_path, get_config_path
from data_handler import Master, init_new_repo, update_database
from util.logger import Logger

In [3]:
# 需要爬取的问题的列表
q_list = [
    # 623539562,
    # 327003589
    # 624316466
    524257471
]
# 存档仓库的地址 # 请修改为您的地址
archiv_path = os.path.join(
    "/media/ywatcher/ExtDisk1/Files/web_archiv_",
    "archiv_1"
)
# archiv_path = "your/path"

In [4]:
# if you have an old version repo
update_database(repo_path=archiv_path)

2.6.0


In [5]:
# 初始化存档仓库，如果存档路径已经存在且不是文件夹，
# 或者存档路径已经存在或者非空，
# 或者存档路径的上一级目录不存在/不是文件夹，
# 会抛出异常，并终止创建仓库。
try:
    init_new_repo(
        repo_path=archiv_path,
    )
except Exception as e:
    print(e)

/media/ywatcher/ExtDisk1/Files/web_archiv_/archiv_1 exists, and is not empty


In [ ]:
from data_handler.database import DataBase
DataBase.init_database(archiv_path)

In [17]:
# 初始化进程间交流使用的队列
q_task = Queue()
q_result = Queue()

如果一个进程 `p` 使用进程队列 `q` 并往里面放入数据，
那么当队列 `q` 在读入大量数据之后，需要将 `q` 中大部
分数据取出后，进程 `p` 才能执行 `.join()`；否则可能
会在 `join()` 时阻塞，此时内部执行的函数是等待 `q` 
拿到它的线程锁（队列 `q` 维护了一个它自己的线程）.<br>
https://stackoverflow.com/questions/71947784/what-does-webdriverwaitdriver-20-mean<br>
https://stackoverflow.com/questions/26869200/python-queue-module-get-stuck



In [7]:
# 根据仓库路径，初始化各个子部分的路径，并读取仓库配置文件
db_path = get_db_path(archiv_repo_path=archiv_path)
vcs_path = get_vcs_path(archiv_repo_path=archiv_path)
repo_config_path = get_config_path(archiv_repo_path=archiv_path)
repo_config = yaml.safe_load(open(repo_config_path, 'r')) 

In [8]:
repo_config

{'owner_email': 'me@email', 'owner_name': 'me'}

In [18]:
# 初始化 selenium web driver 和爬取器
# 初始化 web driver 时，会有相应的窗口
# 打开
# 初始化问题爬取器
question_driver = webdriver.Chrome(drivers["chrome"])
question_parser = QuestionParser(
    queue_put_result=q_result,
    queue_put_task=q_task,
    driver=question_driver,
    logger=Logger("q")
)
# 初始化多个回答爬取器
answer_drivers = []
answer_parsers = []
for i in range(nr_answer_parsers):
    answer_drivers.append(
        webdriver.Chrome(drivers["chrome"])
    )
    answer_parsers.append(
        AnswerParser(
            queue_put_result=q_result,
            queue_get_task=q_task,
            driver=answer_drivers[i],
            logger=Logger("a_{}".format(i))
        )
    )
# 初始化爬取结果处理器
master = Master(
    db_path=db_path,
    git_repo_path=vcs_path
)


In [19]:
# 初始化爬取器和处理器的进程
targets = [
    # target func and args
    (question_parser.start_parsing_list, (q_list,))
] + [
    (answer_parser.start_parsing, [])
    for answer_parser in answer_parsers
] + [
    # (master.start, (config["commiter_name"],config["commiter_email"],q_result))
    (master.start_parse, (
        repo_config['owner_name'],
        repo_config["owner_email"],
        q_result
    ))
]


processes = [
    Process(target=target_func, args=args)
    for target_func, args in targets
]






In [20]:
# 开启爬取进程
# 目前滚动爬取的机制不够完善，可以暂时点开
# 爬取问题的 webdriver 窗口，并手动辅助
# 滚动，否则问题下的答案爬取将过早结束；
# 之后将完善不同的模拟滚动策略
for p in processes[:-1]:
    p.start()

524257471
q: parsing question with id=524257471
q: Scrolled to bottom.
a_0: parsing answer with id=2416068964
a_1: parsing answer with id=2420199852
a_2: parsing answer with id=2414300487
q: parsed 524257471
q: fin
a_2: parsing answer with id=2409004144
a_1: parsing answer with id=2420509046


Traceback (most recent call last):
  File "/home/ywatcher/anaconda3/envs/web/lib/python3.9/multiprocessing/queues.py", line 244, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/home/ywatcher/anaconda3/envs/web/lib/python3.9/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
RecursionError: maximum recursion depth exceeded while pickling an object


a_0: parsing answer with id=2420828250
a_2: parsing answer with id=2423870218
a_1: parsing answer with id=2418549065
a_0: parsing answer with id=2410792129
a_1: parsing answer with id=2480469446
a_0: parsing answer with id=2428001195
a_2: parsing answer with id=2416538542
a_1: parsing answer with id=2480969858
a_0: parsing answer with id=2414576015
a_2: parsing answer with id=2497172256
a_1: parsing answer with id=2420423221
a_0: parsing answer with id=2420660557
a_2: parsing answer with id=2439752004
a_1: parsing answer with id=2417096101
a_0: parsing answer with id=2485879406
a_2: parsing answer with id=2429517879
a_0: parsing answer with id=2421552954
a_2: parsing answer with id=2420192937
a_1: parsing answer with id=2417259863
a_2: parsing answer with id=2416205865
a_0: parsing answer with id=2409174634
a_1: parsing answer with id=2852801503
a_2: stop
a_2: put
a_2: out
a_0: stop
a_0: put
a_0: out
a_1: stop
a_1: put
a_1: out


In [21]:
# 开启处理进程
processes[-1].start()
# 结束所有进程
for p in processes:
    p.join()

In [ ]:
class Controller:
    def __init__(
        self, 
    ):
        self.processes = []
        pass